In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [95]:
#read in state df
df = pd.read_csv("Desktop/JH10States.csv")

In [96]:
df.head(30)

,County,State,Date,Confirmed,Deaths,Combined_Key,cases_per_day
0,Adair,Iowa,1/1/21,606,17,"Adair, Iowa, US",0.0
1,Adair,Iowa,1/2/21,610,17,"Adair, Iowa, US",4.0
2,Adair,Iowa,1/3/21,614,17,"Adair, Iowa, US",4.0
3,Adair,Iowa,1/4/21,616,17,"Adair, Iowa, US",2.0
4,Adair,Iowa,1/5/21,620,17,"Adair, Iowa, US",4.0
5,Adair,Iowa,1/6/21,621,17,"Adair, Iowa, US",1.0
6,Adair,Iowa,1/7/21,636,17,"Adair, Iowa, US",15.0
7,Adair,Iowa,1/8/21,645,17,"Adair, Iowa, US",9.0
8,Adair,Iowa,1/9/21,646,18,"Adair, Iowa, US",1.0
9,Adair,Iowa,1/10/21,653,18,"Adair, Iowa, US",7.0


In [97]:
#remove unassigned county
df = df[df.County != 'Unassigned']

In [98]:
df.shape

(715405, 7)

In [99]:
#remove county with a lot of negative number of cases
df = df[df.Combined_Key != 'Anderson, Texas, US']

In [100]:
df.shape

(714733, 7)

In [101]:
#remove county with a lot of negative number of cases
df = df[df.Combined_Key != 'Nash, North Carolina, US']

In [102]:
df.shape

(714061, 7)

In [103]:
#remove non county data
df = df[df.County != 'Out of CA']
df.shape

(713648, 7)

In [104]:
#remove non county data
df = df[df.County != 'Out of IL']
df = df[df.County != 'Out of KS']
df.shape

(712319, 7)

In [105]:
#remove county with a lot of negative number of cases
df = df[df.Combined_Key != 'Waller, Texas, US']
df.shape

(711647, 7)

In [106]:
Total = df['cases_per_day'].sum()
print(Total)

26448128.843856186


In [107]:
#read in df with CDC region for each state
dfregion = pd.read_csv("Desktop/CDCRegion.csv")

In [108]:
df['Date'] = pd.to_datetime(df['Date'])

In [109]:
#sum cases by weeks in CDC data (on saturdays)
df = df.groupby(['County','State', pd.Grouper(key='Date', freq='W-SAT')])['cases_per_day'].sum().reset_index()

In [110]:
df.head()

,County,State,Date,cases_per_day
0,Adair,Iowa,2021-01-02,4.0
1,Adair,Iowa,2021-01-09,36.0
2,Adair,Iowa,2021-01-16,36.0
3,Adair,Iowa,2021-01-23,13.0
4,Adair,Iowa,2021-01-30,7.0


In [111]:
df['cases_per_day'].sum() 

26448128.843856182

In [112]:
dfregion.head()

,State Name,CDC Region
0,Alabama,4
1,Alaska,10
2,Arizona,9
3,Arkansas,6
4,California,9


In [113]:
#left join on date to get weekly cumulative cases for the different variants
df=pd.merge(df,dfregion,left_on=['State'],right_on=['State Name'])

In [114]:
#read in df with CDC region variant proportion data for each region
dfvar = pd.read_csv("Downloads/CDCVariantProportionsDeDuped.csv")

In [115]:
dfvar.head()

,usa_or_hhsregion,week_ending,variant,share,share_hi,share_lo,nchs_or_count_flag,modeltype,time_interval,published_date
0,1,1/2/21,B.1.2,0.286312,0.422580,0.180268,NaN,weighted,biweekly,5/4/21
1,1,1/2/21,B.1.1.7,0.000000,NaN,NaN,NaN,weighted,biweekly,5/4/21
2,1,1/2/21,B.1.429,0.001683,0.035162,0.000078,NaN,weighted,biweekly,5/4/21
3,1,1/2/21,B.1.596,0.051787,0.118745,0.021658,NaN,weighted,biweekly,5/4/21
4,1,1/2/21,B.1.427,0.000000,NaN,NaN,NaN,weighted,biweekly,5/4/21


In [116]:
#convert date column to datetime
dfvar['week_ending'] = pd.to_datetime(dfvar['week_ending'])

In [117]:
#drop unnecessary columns
dfvar = dfvar.drop(['share_hi', 'share_lo', 'nchs_or_count_flag', 'modeltype'], axis=1)

In [118]:
#inner join on region and date to merge DFs
dfjoin=pd.merge(df,dfvar,left_on=['Date','CDC Region'] ,right_on=['week_ending', 'usa_or_hhsregion'])

In [119]:
dfjoin.head()

,County,State,Date,cases_per_day,State Name,CDC Region,usa_or_hhsregion,week_ending,variant,share,time_interval,published_date
0,Adair,Iowa,2021-01-02,4.0,Iowa,7,7,2021-01-02,B.1.2,0.565277,biweekly,5/4/21
1,Adair,Iowa,2021-01-02,4.0,Iowa,7,7,2021-01-02,B.1.1.7,0.000000,biweekly,5/4/21
2,Adair,Iowa,2021-01-02,4.0,Iowa,7,7,2021-01-02,B.1.429,0.024311,biweekly,5/4/21
3,Adair,Iowa,2021-01-02,4.0,Iowa,7,7,2021-01-02,B.1.596,0.041531,biweekly,5/4/21
4,Adair,Iowa,2021-01-02,4.0,Iowa,7,7,2021-01-02,B.1.427,0.001590,biweekly,5/4/21


In [120]:
#multiply the variant share by the # new cases per week
dfjoin['var_cases_per_week'] = dfjoin.share * dfjoin.cases_per_day

In [121]:
dfjoin['var_cases_per_week'].sum() 

25242726.18290027

In [122]:
#drop unnecessary columns
dfjoin = dfjoin.drop(['State Name', 'usa_or_hhsregion', 'week_ending', 'published_date', 'time_interval'], axis=1)

In [123]:
#creates df with only max values for var cases per week grouped by county, state, and variant type
dfmax=dfjoin.loc[dfjoin.groupby(['County','State', 'variant']).var_cases_per_week.idxmax().values]

In [124]:
dfmax.head(20)

,County,State,Date,cases_per_day,CDC Region,variant,share,var_cases_per_week
9504,Adair,Iowa,2021-01-30,7.0,7,AY.1,0.000000,0.000000
9505,Adair,Iowa,2021-01-30,7.0,7,AY.2,0.000000,0.000000
4757,Adair,Iowa,2021-01-16,36.0,7,B.1,0.031201,1.123236
4758,Adair,Iowa,2021-01-16,36.0,7,B.1.1.519,0.015908,0.572675
224235,Adair,Iowa,2022-01-22,48.0,7,B.1.1.529,0.368590,17.692310
55838,Adair,Iowa,2021-05-01,11.0,7,B.1.1.7,0.805537,8.860910
4752,Adair,Iowa,2021-01-16,36.0,7,B.1.2,0.491552,17.695877
4761,Adair,Iowa,2021-01-16,36.0,7,B.1.234,0.068710,2.473556
4760,Adair,Iowa,2021-01-16,36.0,7,B.1.243,0.005834,0.210014
48714,Adair,Iowa,2021-04-17,5.0,7,B.1.351,0.018320,0.091602


In [157]:
#drops rows where var cases per week is less than 2
dffirst = dfjoin[dfjoin["var_cases_per_week"] > 2]

In [158]:
#creates df with only min values for date state and variant type to get first week variant reached 1+ cases
dffirst=dffirst.loc[dffirst.groupby(['County','State', 'variant']).Date.idxmin().values]

In [159]:
dffirst.head(30)

,County,State,Date,cases_per_day,CDC Region,variant,share,var_cases_per_week
201960,Adair,Iowa,2021-12-18,30.000000,7,B.1.1.529,0.074646,2.239368
34454,Adair,Iowa,2021-03-20,10.000000,7,B.1.1.7,0.340360,3.403595
0,Adair,Iowa,2021-01-02,4.000000,7,B.1.2,0.565277,2.261108
4761,Adair,Iowa,2021-01-16,36.000000,7,B.1.234,0.068710,2.473556
4755,Adair,Iowa,2021-01-16,36.000000,7,B.1.596,0.108660,3.911764
90586,Adair,Iowa,2021-06-26,3.000000,7,B.1.617.2,0.875770,2.627310
206417,Adair,Iowa,2021-12-25,42.000000,7,BA.1.1,0.144885,6.085166
286608,Adair,Iowa,2022-04-30,4.000000,7,BA.2,0.577110,2.308439
326704,Adair,Iowa,2022-07-02,11.000000,7,BA.2.12.1,0.248040,2.728439
375713,Adair,Iowa,2022-09-17,17.000000,7,BA.4.6,0.124498,2.116470


In [160]:
dffirst.rename(columns={'Date':'first_date', 'share':'share_first', 'var_cases_per_week':'first_cases_per_week'}, inplace=True)

In [161]:
dfmax.rename(columns={'Date':'max_date', 'share':'share_max', 'var_cases_per_week':'max_cases_per_week'}, inplace=True)

In [162]:
#inner join on region and date to merge DFs
df_first_max=pd.merge(dfmax,dffirst,left_on=['County','State','variant'] ,right_on=['County','State', 'variant'])

In [163]:
df_first_max.head()

,County,State,max_date,cases_per_day_x,CDC Region_x,variant,share_max,max_cases_per_week,first_date,cases_per_day_y,CDC Region_y,share_first,first_cases_per_week
0,Adair,Iowa,2022-01-22,48.0,7,B.1.1.529,0.368590,17.692310,2021-12-18,30.0,7,0.074646,2.239368
1,Adair,Iowa,2021-05-01,11.0,7,B.1.1.7,0.805537,8.860910,2021-03-20,10.0,7,0.340360,3.403595
2,Adair,Iowa,2021-01-16,36.0,7,B.1.2,0.491552,17.695877,2021-01-02,4.0,7,0.565277,2.261108
3,Adair,Iowa,2021-01-16,36.0,7,B.1.234,0.068710,2.473556,2021-01-16,36.0,7,0.068710,2.473556
4,Adair,Iowa,2021-01-16,36.0,7,B.1.596,0.108660,3.911764,2021-01-16,36.0,7,0.108660,3.911764


In [164]:
#drop unnecessary columns
df_first_max = df_first_max.drop(['cases_per_day_x', 'CDC Region_x', 'cases_per_day_y'], axis=1)

In [165]:
df_first_max.rename(columns={'CDC Region_y':'CDC_Region'}, inplace=True)

In [166]:
df_first_max.head()

,County,State,max_date,variant,share_max,max_cases_per_week,first_date,CDC_Region,share_first,first_cases_per_week
0,Adair,Iowa,2022-01-22,B.1.1.529,0.368590,17.692310,2021-12-18,7,0.074646,2.239368
1,Adair,Iowa,2021-05-01,B.1.1.7,0.805537,8.860910,2021-03-20,7,0.340360,3.403595
2,Adair,Iowa,2021-01-16,B.1.2,0.491552,17.695877,2021-01-02,7,0.565277,2.261108
3,Adair,Iowa,2021-01-16,B.1.234,0.068710,2.473556,2021-01-16,7,0.068710,2.473556
4,Adair,Iowa,2021-01-16,B.1.596,0.108660,3.911764,2021-01-16,7,0.108660,3.911764


In [167]:
#calcualte difference between week of first and max # of cases (to create target variable)
df_first_max['diff_weeks'] = df_first_max['max_date'] - df_first_max['first_date']
df_first_max['diff_weeks']=df_first_max['diff_weeks']/np.timedelta64(1,'W')

In [168]:
df_first_max.to_csv('Downloads/df_first_max_county.csv')

In [136]:
dfjoin.to_csv('Downloads/dfjoinCounty.csv')

In [137]:
dfjoin.shape

(1450830, 8)

In [169]:
#read in county transportation df
dftransport = pd.read_csv("Downloads/County_transportation.csv")

In [170]:
dftransport.head()

,County Name,State Name,Primary and Commercial Airports,Non-Commercial -Civil Public Use Airports and Seaplane base,Number of Bridges,Number of business establishments,Percent of resident workers who commute by transit,Number of residents,Miles freight railroad,Miles passenger railroad
0,Autauga,Alabama,0.0,1.0,114,869,0.000782,"55,050",50.7,0.0
1,Baldwin,Alabama,0.0,4.0,360,"5,384",0.002143,"199,510",30.4,0.0
2,Barbour,Alabama,0.0,2.0,193,455,0.003984,"26,615",14.7,0.0
3,Bibb,Alabama,0.0,1.0,126,284,0.004710,"22,570",25.1,5.5
4,Blount,Alabama,0.0,1.0,183,698,0.001639,"57,705",13.1,0.0


In [171]:
#left join on county DFs
df_first_max=pd.merge(df_first_max,dftransport,left_on=['County','State'], right_on=['County Name','State Name'], how = 'left')

,County,State,max_date,variant,share_max,max_cases_per_week,first_date,CDC_Region,share_first,first_cases_per_week,...,County Name,State Name,Primary and Commercial Airports,Non-Commercial -Civil Public Use Airports and Seaplane base,Number of Bridges,Number of business establishments,Percent of resident workers who commute by transit,Number of residents,Miles freight railroad,Miles passenger railroad
0,Adair,Iowa,2021-01-16,B.1,0.031201,1.123236,2021-01-16,7,0.031201,1.123236,...,Adair,Iowa,0.0,1.0,317,200,0.005464,"7,330",7.7,0.0
1,Adair,Iowa,2022-01-22,B.1.1.529,0.368590,17.692310,2021-12-18,7,0.074646,2.239368,...,Adair,Iowa,0.0,1.0,317,200,0.005464,"7,330",7.7,0.0
2,Adair,Iowa,2021-05-01,B.1.1.7,0.805537,8.860910,2021-02-13,7,0.028336,1.161792,...,Adair,Iowa,0.0,1.0,317,200,0.005464,"7,330",7.7,0.0
3,Adair,Iowa,2021-01-16,B.1.2,0.491552,17.695877,2021-01-02,7,0.565277,2.261108,...,Adair,Iowa,0.0,1.0,317,200,0.005464,"7,330",7.7,0.0
4,Adair,Iowa,2021-01-16,B.1.234,0.068710,2.473556,2021-01-16,7,0.068710,2.473556,...,Adair,Iowa,0.0,1.0,317,200,0.005464,"7,330",7.7,0.0


In [172]:
df_first_max['Number of residents'] = df_first_max['Number of residents'].str.replace(',','')
df_first_max = df_first_max.astype({'Number of residents':'int'})
#drops rows where county population is less than 10k
df_first_max2 = df_first_max[df_first_max["Number of residents"] > 10000]

In [173]:
df_first_max2.head()

,County,State,max_date,variant,share_max,max_cases_per_week,first_date,CDC_Region,share_first,first_cases_per_week,...,County Name,State Name,Primary and Commercial Airports,Non-Commercial -Civil Public Use Airports and Seaplane base,Number of Bridges,Number of business establishments,Percent of resident workers who commute by transit,Number of residents,Miles freight railroad,Miles passenger railroad
12,Adams,Illinois,2021-01-16,B.1,0.036368,11.055908,2021-01-16,5,0.036368,11.055908,...,Adams,Illinois,1.0,0.0,314,"1,825",0.005127,66950,40.2,29.5
13,Adams,Illinois,2021-01-16,B.1.1.519,0.009185,2.792146,2021-01-16,5,0.009185,2.792146,...,Adams,Illinois,1.0,0.0,314,"1,825",0.005127,66950,40.2,29.5
14,Adams,Illinois,2022-01-15,B.1.1.529,0.410995,585.668479,2021-12-11,5,0.026623,11.767301,...,Adams,Illinois,1.0,0.0,314,"1,825",0.005127,66950,40.2,29.5
15,Adams,Illinois,2021-04-24,B.1.1.7,0.705996,53.655663,2021-02-06,5,0.031653,3.450227,...,Adams,Illinois,1.0,0.0,314,"1,825",0.005127,66950,40.2,29.5
16,Adams,Illinois,2021-01-16,B.1.2,0.569843,173.232397,2021-01-16,5,0.569843,173.232397,...,Adams,Illinois,1.0,0.0,314,"1,825",0.005127,66950,40.2,29.5


In [174]:
#read in county demographic df
dfdemo = pd.read_csv("Downloads/County Demographics.csv")

In [175]:
dfdemo.head()

,County,State,% HS Diploma,Total people in poverty (%),Poverty under 18 (%),Unempl rate,median Income
0,Adair,Iowa,0.805,0.102,0.120,0.038,55265.0
1,Adams,Iowa,0.827,0.108,0.143,0.037,56291.0
2,Allamakee,Iowa,0.818,0.108,0.160,0.050,55320.0
3,Appanoose,Iowa,0.816,0.109,0.166,0.052,49737.0
4,Audubon,Iowa,0.812,0.146,0.197,0.034,56338.0


In [176]:
#left join on county DFs
df_first_max2=pd.merge(df_first_max2,dfdemo,left_on=['County','State'], right_on=['County','State'], how = 'left')

In [177]:
df_first_max2.head()

,County,State,max_date,variant,share_max,max_cases_per_week,first_date,CDC_Region,share_first,first_cases_per_week,...,Number of business establishments,Percent of resident workers who commute by transit,Number of residents,Miles freight railroad,Miles passenger railroad,% HS Diploma,Total people in poverty (%),Poverty under 18 (%),Unempl rate,median Income
0,Adams,Illinois,2021-01-16,B.1,0.036368,11.055908,2021-01-16,5,0.036368,11.055908,...,"1,825",0.005127,66950,40.2,29.5,0.748,0.107,0.132,0.064,57434.0
1,Adams,Illinois,2021-01-16,B.1.1.519,0.009185,2.792146,2021-01-16,5,0.009185,2.792146,...,"1,825",0.005127,66950,40.2,29.5,0.748,0.107,0.132,0.064,57434.0
2,Adams,Illinois,2022-01-15,B.1.1.529,0.410995,585.668479,2021-12-11,5,0.026623,11.767301,...,"1,825",0.005127,66950,40.2,29.5,0.748,0.107,0.132,0.064,57434.0
3,Adams,Illinois,2021-04-24,B.1.1.7,0.705996,53.655663,2021-02-06,5,0.031653,3.450227,...,"1,825",0.005127,66950,40.2,29.5,0.748,0.107,0.132,0.064,57434.0
4,Adams,Illinois,2021-01-16,B.1.2,0.569843,173.232397,2021-01-16,5,0.569843,173.232397,...,"1,825",0.005127,66950,40.2,29.5,0.748,0.107,0.132,0.064,57434.0


In [178]:
Variants = ['B.1.1.529', 'B.1.1.7', 'B.1.526', 'B.1.617.2', 'BA.1.1', 'BA.2', 'BA.2.12.1', 'BA.4', 'BA.4.6', 'BA.5']
# drop rows that don't contain variants from above list
df_first_max2=df_first_max2[df_first_max2.variant.isin(Variants) == True]

In [179]:
#create region dummy variables
regiondummies = pd.get_dummies(df_first_max2.CDC_Region)
df_first_max2=pd.concat([df_first_max2, regiondummies], axis=1)

In [180]:
#create variant dummy variables
variantdummies = pd.get_dummies(df_first_max2.variant)
df_first_max2=pd.concat([df_first_max2, variantdummies], axis=1)

In [181]:
#add feature of quarter variant stated
df_first_max2['Quarter_first'] = df_first_max2['first_date'].dt.quarter

In [182]:
#add feature or quarter variant maxed # of weekly cases
df_first_max2['Quarter_max'] = df_first_max2['max_date'].dt.quarter

In [183]:
df_first_max2.to_csv('Downloads/Alldatacounty4.csv')

In [80]:
dfjoin.to_csv('Downloads/dfjoinCounty2.csv')

In [184]:
df_first_max2.shape

(7275, 43)

In [85]:
dfjoin.to_csv('Downloads/dfjoinC3.csv')

,County,State,Date,cases_per_day,CDC Region,variant,share,var_cases_per_week
1026530,Mono,California,2021-01-02,16.0,9,B.1.2,0.350053,5.600840
1026531,Mono,California,2021-01-02,16.0,9,B.1.1.7,0.000000,0.000000
1026532,Mono,California,2021-01-02,16.0,9,B.1.429,0.102753,1.644045
1026533,Mono,California,2021-01-02,16.0,9,B.1.596,0.050335,0.805354
1026534,Mono,California,2021-01-02,16.0,9,B.1.427,0.126316,2.021061
